In [15]:
import pickle
import os
from datetime import datetime
import pandas as pd
import numpy as np

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [16]:
model_path = r"C:\Users\Akshay\Documents\Project\Models\xgboost_regressor_r2_0_916_v1.pkl"
model = pickle.load(open(model_path, "rb"))   # rb :- Read Binary.

In [17]:
data = [
    [ 1.07807977,  0.22192612,  0.77468297,  1.82005277, -1.26200749,
       -0.25194272, -0.12916836, -0.17535463, -0.22754885, -5.47803337,
       -1.21546253,  1.00873202,  0.30459857,  1.74132735, -0.58152447,
       -0.57866842, -0.39840954, -0.40773843, -0.41815465, -0.40773843,
       -0.40559398, -0.41815465],
    [-0.65432992,  1.24504573,  0.23697162, -0.5001855 ,  0.56154562,
       -0.28664608, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
        0.94712104,  0.14077827,  0.30459857, -0.57427456,  1.71961809,
       -0.57866842, -0.39840954, -0.40773843, -0.41815465, -0.40773843,
        2.4655198 , -0.41815465]
   
]


prediction = model.predict(data)
list(prediction)

[-134.08298, 1242.9778]

## Load SC

In [18]:
sc_dump_path = r"C:\Users\Akshay\Documents\Project\Models\sc.pkl"

sc = pickle.load(open(sc_dump_path, "rb"))

In [19]:
sc

StandardScaler()

## User Input

In [29]:
date = "15/06/2023"
hour = 18
temperature = 25
humidity = 55
wind_speed = 67
visibility = 80
solar_radiation = 0.0
rainfall = 0.0
snowfall = 0.2
seasons = "Winter"
holiday = "No Holiday"
functioning_day = "No"

## Converting User Data into Model Consumable Format

In [30]:
from datetime import datetime

holiday_dic = {"No Holiday" : 0, "Holiday" : 1}
functioning_day = {"No" : 0, "Yes" : 1}

holiday_dic["holiday"] = 0
functioning_day["functioning_day"] = 1

def get_string_to_datetime(date):
    dt = datetime.strptime(date, "%d/%m/%Y")
    return {"day":dt.day, "month":dt.month, "year":dt.year, "weekday":dt.strftime("%A")}

str_to_date = get_string_to_datetime(date)
str_to_date

{'day': 15, 'month': 6, 'year': 2023, 'weekday': 'Thursday'}

In [31]:
u_input_list = [hour, temperature, humidity, wind_speed, visibility, solar_radiation, rainfall, snowfall,
                holiday_dic.get('holiday'), functioning_day.get('functioning_day'),
                str_to_date["day"], str_to_date["month"], str_to_date["year"]]
                
features_name = ['Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)', 
                 'Rainfall (mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day', 'Month', 'Year']

df_u_input = pd.DataFrame([u_input_list], columns = features_name)
df_u_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall (mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,18,25,55,67,80,0.0,0.0,0.2,0,1,15,6,2023


In [32]:
def season_to_df(seasons):   
    seasons_cols = ['Spring', 'Summer', 'Winter']
    seasons_data = np.zeros((1, len(seasons_cols)), dtype="int")

    df_seasons = pd.DataFrame(seasons_data, columns = seasons_cols)
    if seasons in seasons_cols:
        df_seasons[seasons]=1
    return df_seasons

df_season=season_to_df(seasons)
df_season

,Spring,Summer,Winter
0,0,0,1


In [33]:
def days_df(weekday):
    days_names = ['Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    days_name_data = np.zeros((1, len(days_names)), dtype="int")
    
    df_days = pd.DataFrame(days_name_data, columns = days_names)
    
    if weekday in days_names:
        df_days[weekday]=1
        return df_days
    
df_days = days_df(str_to_date["weekday"])
df_days

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,0,1,0,0


In [34]:
['Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(°C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Holiday',
       'Functioning Day', 'Day', 'Month', 'Year', 'Spring', 'Summer', 'Winter',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']

['Hour',
 'Temperature(°C)',
 'Humidity(%)',
 'Wind speed (m/s)',
 'Visibility (10m)',
 'Dew point temperature(°C)',
 'Solar Radiation (MJ/m2)',
 'Rainfall(mm)',
 'Snowfall (cm)',
 'Holiday',
 'Functioning Day',
 'Day',
 'Month',
 'Year',
 'Spring',
 'Summer',
 'Winter',
 'Monday',
 'Saturday',
 'Sunday',
 'Thursday',
 'Tuesday',
 'Wednesday']

In [35]:
df_for_pred = pd.concat([df_u_input, df_season, df_days], axis=1)
df_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall (mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,18,25,55,67,80,0.0,0.0,0.2,0,1,...,2023,0,0,1,0,0,0,1,0,0


In [36]:
sc_data_for_pred = sc.transform(df_for_pred)
sc_data_for_pred

array([[ 0.93371229,  1.01214045, -0.15409119, 63.11301355, -2.24303289,
        -0.65681529, -0.12916836,  0.29083855, -0.22754885,  0.18254726,
        -0.07726065, -0.14853965, 18.24263884, -0.57427456, -0.58152447,
         1.72810536, -0.39840954, -0.40773843, -0.41815465,  2.45255275,
        -0.40559398, -0.41815465]])

## <font color='blue'> Prediction in Deployment </font>

In [37]:
print(f"Rented Bike Demand on Date: {date}, and Time: {hour} is : {round(model.predict(sc_data_for_pred).tolist()[0])}")

Rented Bike Demand on Date: 15/06/2023, and Time: 18 is : 1610
